# Ensemble methods: Tree Bagging; Random Forests; Adaboost

We talked a bit in passing about a few ensemble methods when we talked about trees etc. Let's take some time to use them! We'll go over both the sklearn implementations, and try implementing both ourselves. In the 'do it yourself' part, I'll give you a single iteration, it is your job to put it together ;)

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.datasets import load_boston
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier 

# For producing decision tree diagrams.
from IPython.core.display import Image, display
from sklearn.externals.six import StringIO


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Today, we'll instead use the famous *boston housing data* to try out ensemble methods. We are going to make the output binary, so that we can focus on classification. We'll do regression later.

In [2]:
# load the boston housing data
boston = load_boston()
X, Y = boston.data, boston.target

# binarize the output so it is now a classification task
Y = 1 * (Y > np.median(Y))

# Shuffle the data, but make sure that the features and accompanying labels stay in sync.
np.random.seed(0)
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

# Split into train and test.
train_data, train_labels = X[:350], Y[:350]
test_data, test_labels = X[350:], Y[350:]

For the following questions, you might find this function useful to print out the tree. If you want to try a graphical way, look into this function:
http://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html

The below function prints out a 'pseudocode' version of the tree, in terms of if-else statements.

In [3]:
def get_code(tree, feature_names):
        left      = tree.tree_.children_left
        right     = tree.tree_.children_right
        threshold = tree.tree_.threshold
        features  = [feature_names[i] for i in tree.tree_.feature]
        value = tree.tree_.value

        def recurse(left, right, threshold, features, node):
                if (threshold[node] != -2):
                        print ("if ( " + features[node] + " <= " + str(threshold[node]) + " ) {")
                        if left[node] != -1:
                                recurse (left, right, threshold, features,left[node])
                        print ("} else {")
                        if right[node] != -1:
                                recurse (left, right, threshold, features,right[node])
                        print ("}")
                else:
                        print ("return " + str(value[node]))
        recurse(left, right, threshold, features, 0)

# example call:
#get_code(dt, boston.feature_names)

## Ensemble Methods!

Let's explore what sklearn has in terms of ensemble methods. There are two interesting ones we can use right now, adaboost and random forests. We'll start by using the sklearn ones, then try implementing random forests ourselves!

Be sure to reference the documentation at:  http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Let's start with just executing some sklearn functions:

In [4]:
dt = DecisionTreeClassifier(criterion="entropy", splitter="best", random_state=0)
dt.fit(train_data, train_labels)

print ('Accuracy (a decision tree):', dt.score(test_data, test_labels))

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_data, train_labels)

print ('Accuracy (a random forest):', rfc.score(test_data, test_labels))

abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate=0.1)

abc.fit(train_data, train_labels)
print ('Accuracy (adaboost with decision trees):', abc.score(test_data, test_labels))

Accuracy (a decision tree): 0.8782051282051282
Accuracy (a random forest): 0.9102564102564102
Accuracy (adaboost with decision trees): 0.9102564102564102


In [5]:
# lets see if that tree code works!
get_code(dt, boston.feature_names)

if ( LSTAT <= 7.684999942779541 ) {
if ( PTRATIO <= 21.5 ) {
return [[ 0. 94.]]
} else {
return [[1. 0.]]
}
} else {
if ( LSTAT <= 14.045000076293945 ) {
if ( RM <= 6.315499782562256 ) {
if ( INDUS <= 13.360000133514404 ) {
if ( CRIM <= 0.05717500112950802 ) {
return [[14.  0.]]
} else {
if ( CRIM <= 0.5821950137615204 ) {
if ( TAX <= 404.5 ) {
if ( INDUS <= 4.2200000286102295 ) {
return [[0. 4.]]
} else {
if ( AGE <= 28.049999237060547 ) {
return [[0. 3.]]
} else {
if ( RM <= 5.92300009727478 ) {
return [[8. 0.]]
} else {
if ( AGE <= 45.39999961853027 ) {
if ( TAX <= 255.0 ) {
return [[1. 0.]]
} else {
return [[0. 5.]]
}
} else {
if ( B <= 395.9150085449219 ) {
if ( NOX <= 0.47050000727176666 ) {
return [[4. 0.]]
} else {
if ( DIS <= 2.746250033378601 ) {
return [[2. 0.]]
} else {
return [[0. 5.]]
}
}
} else {
return [[8. 0.]]
}
}
}
}
}
} else {
return [[4. 0.]]
}
} else {
return [[7. 0.]]
}
}
} else {
if ( B <= 390.7099914550781 ) {
if ( NOX <= 0.6010000109672546 ) {
return [[4. 0.]]

It looks like ensemble methods do well, both do better than a single tree. Before moving on, try playing arond with some of the parameters, such as:

n_estimators in RandomForestClassifier


n_estimators and learning_rate AdaBoostClassifier

Why do the methods behave as they when you tweak the parameters?

### Random forests

Random forests are combinations of many decision trees. Let's start with a slightly simplified version: **tree bagging**. Here is a simple algorithm for tree bagging:

1. Set B (number of trees to make)
2. Repeat B times:
  1. Draw N random samples from training data, with replacement, where N is the number of training data points
  2. Fit a decision tree to this re-sampled data
  3. Store the predictions from this decision tree on the test data
3. As the final predictions on the test data, use the majority vote classification for the predictions above

Below, I've given you an implementation of a single iteration of the main loop above. Complete the algorthim by (1) adding the repeated B resampling and fitting (2) implementing step 3 above, the final predictions from tree bagging.

Once you've done that, does bagging do better than a single tree?

In [6]:
np.random.seed(1)

# the following can be replaced in a single line with:
# np.random.choice(range(n), size=n, replace=True)
# but I've given the explicit code so you can learn a bit :-)
def bs_sample_index(n):
    bootstrap_sample_index = np.random.rand(n)
    bootstrap_sample_index = np.floor(bootstrap_sample_index * n)
    bootstrap_sample_index = bootstrap_sample_index.astype("int")
    
    return bootstrap_sample_index

# a single iteration of tree bagging

bootstrap_sample_index = bs_sample_index(train_data.shape[0])

bs_data = train_data[bootstrap_sample_index, :]
bs_labels = train_labels[bootstrap_sample_index]
    
# without max_feature restriction this is 'tree bagging'
bs_tree = DecisionTreeClassifier(criterion="entropy", splitter="best")
bs_tree.fit(bs_data, bs_labels)

bs_tree_preds = bs_tree.predict(test_data)

print (sum(bs_tree_preds == test_labels) / len(test_labels))  # manual scoring
print (bs_tree.score( test_data, test_labels))

0.8717948717948718
0.8717948717948718


Now, we are ready to do **random forests**. Random forests add the twist of subsampling features at each node. Typically, we take p' = sqrt(p) features. DecisionTreeClassifier implements with through the *max_features*, check out the documentation. A simple change to your above code should give you random forests.

Does random forests do better than tree bagging?

Note: you can also use trees, tree bagging, and random forests for regression! Now, the original data is a regression problem so just reload the data, and to do all of these ideas using trees, you need only use DecisionTreeRegressor instead of DecisionTreeClassifier; see:
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

As a bonus, try implementing trees, tree bagging, and random forests for regression.

## AdaBoost

check this out

https://www.analyticsvidhya.com/blog/2015/11/quick-introduction-boosting-algorithms-machine-learning/

Adaboost is another popular ensemble method. It operates roughly in the following way:

1. Initialize a set of weights over the data by weighting each datapoint equally. Set a number of learning rounds B
2. Do the following B times:
  1. *Train a simple classifier on the data*, using the current set of weights.
  2. *Compute the overall error rate on the **training data** *, weighted by the current set of weights, of the classifier, **save this error rate**.
  3. *Update the distribution of weights*, giving more importance to datapoints we misclassified
3. Combine the classifiers of all the loop iterations together, weighted by a function (denoted error_rate_alpha below) of their error rates.

Below, a single iteration of adaboost is implemented. 

Before extending it, check out how we update the weights. How does this emphasize errors made in the previous round?

Now, extend it so it actually does multiple learning rounds.

In [7]:
# convert labels into +/- 1; this is just how adaboost likes it :)
train_labels_pm = train_labels * 2 - 1
test_labels_pm = test_labels * 2 - 1

# initialize with equal weights on each data point
data_weights = np.ones(train_data.shape[0]).astype("float") / float(train_data.shape[0])

# here begins a single learning round
bdtc = DecisionTreeClassifier(max_depth=1,criterion="entropy", splitter="best")
bdtc.fit(train_data, train_labels_pm, sample_weight=data_weights)

# you'll need to save bdtc_predictions each iteration of the loop to do final training set predictions
bdtc_predictions = bdtc.predict(train_data)

# you'll need to save bdtc_predictions_test each iteration of the loop to do final test set predictions
bdtc_predictions_test = bdtc.predict(test_data)

    
bdtc_weighted_error_rate = np.sum(data_weights * (1 * (bdtc_predictions != train_labels_pm)).astype("float"))

# you'll need to save error_rate_alpha each iteration of the loop to do all final predictions
error_rate_alpha = np.log((1 - bdtc_weighted_error_rate) / bdtc_weighted_error_rate) / 2
    
# reweighting: how does this emphasize errors?    
data_weights_updated = data_weights * np.exp(-1 * error_rate_alpha * bdtc_predictions * train_labels_pm)
data_weights_updated = data_weights_updated / sum(data_weights_updated)
data_weights = data_weights_updated

In [8]:
bdtc.score(test_data, test_labels_pm)
#sum(bdtc_predictions_test == test_labels) / len(test_labels)

0.8269230769230769

Note that the above algorithm is a *meta algorithm*, meaning we can use any kind of classifier, not just decision trees in the above. Try using another one. Does that change the results?

Note that the above is a bit adapted to a classification loss, we'd need to use something like L2 boosting for regression, that is beyond the scope of this notebook.